In [5]:
import requests
import datetime
import time


In [7]:
API_KEY ="oDYnBLKa3kwNVn3Wecge0EyQGvjVkMJ6ggAklLBw"

import requests
url= f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-05-01&end_date=2025-05-07&api_key={API_KEY}"
response= requests.get(url)
response
data= response.json()
data

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2025-05-07&end_date=2025-05-13&detailed=false&api_key=oDYnBLKa3kwNVn3Wecge0EyQGvjVkMJ6ggAklLBw',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2025-04-25&end_date=2025-05-01&detailed=false&api_key=oDYnBLKa3kwNVn3Wecge0EyQGvjVkMJ6ggAklLBw',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2025-05-01&end_date=2025-05-07&detailed=false&api_key=oDYnBLKa3kwNVn3Wecge0EyQGvjVkMJ6ggAklLBw'},
 'element_count': 106,
 'near_earth_objects': {'2025-05-01': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3373919?api_key=oDYnBLKa3kwNVn3Wecge0EyQGvjVkMJ6ggAklLBw'},
    'id': '3373919',
    'neo_reference_id': '3373919',
    'name': '(2007 GW4)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3373919',
    'absolute_magnitude_h': 20.6,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2016299194,
      'estimated_diameter_max': 0.4508582062},
     'met

In [11]:
api_key = "oDYnBLKa3kwNVn3Wecge0EyQGvjVkMJ6ggAklLBw"

# Start with 7-day range
start_date = datetime.datetime.strptime("2024-05-01", "%Y-%m-%d")
end_date = start_date + datetime.timedelta(days=6)

# URL template
base_url = "https://api.nasa.gov/neo/rest/v1/feed"


In [12]:
def extract_fields(neo, approach_data):
    return {
        "id": neo.get("id"),
        "neo_reference_id": neo.get("neo_reference_id"),
        "name": neo.get("name"),
        "absolute_magnitude_h": neo.get("absolute_magnitude_h"),
        "estimated_diameter_min_km": neo["estimated_diameter"]["kilometers"]["estimated_diameter_min"],
        "estimated_diameter_max_km": neo["estimated_diameter"]["kilometers"]["estimated_diameter_max"],
        "is_potentially_hazardous_asteroid": neo.get("is_potentially_hazardous_asteroid"),
        "close_approach_date": approach_data.get("close_approach_date"),
        "relative_velocity_kmph": float(approach_data["relative_velocity"]["kilometers_per_hour"]),
        "astronomical": float(approach_data["miss_distance"]["astronomical"]),
        "miss_distance_km": float(approach_data["miss_distance"]["kilometers"]),
        "miss_distance_lunar": float(approach_data["miss_distance"]["lunar"]),
        "orbiting_body": approach_data.get("orbiting_body")
    }


In [13]:
all_asteroids = []
max_records = 10000

while len(all_asteroids) < max_records:
    # Format dates to string
    s_date_str = start_date.strftime("%Y-%m-%d")
    e_date_str = end_date.strftime("%Y-%m-%d")
    
    # Build API URL
    url = f"{base_url}?start_date={s_date_str}&end_date={e_date_str}&api_key={api_key}"
    
    # Fetch data from API
    response = requests.get(url)
    data = response.json()

    # Process each day in the response
    for date_key in data.get("near_earth_objects", {}):
        for neo in data["near_earth_objects"][date_key]:
            for approach in neo.get("close_approach_data", []):
                record = extract_fields(neo, approach)
                all_asteroids.append(record)
                
                if len(all_asteroids) >= max_records:
                    break
            if len(all_asteroids) >= max_records:
                break
        if len(all_asteroids) >= max_records:
            break
    
    # Move to next date range
    start_date += datetime.timedelta(days=7)
    end_date += datetime.timedelta(days=7)
    
    print(f"Collected {len(all_asteroids)} records so far...")

    # Optional: short pause to avoid overloading the API
    time.sleep(1)


Collected 119 records so far...
Collected 244 records so far...
Collected 351 records so far...
Collected 452 records so far...
Collected 568 records so far...
Collected 670 records so far...
Collected 762 records so far...
Collected 864 records so far...
Collected 973 records so far...
Collected 1072 records so far...
Collected 1163 records so far...
Collected 1250 records so far...
Collected 1333 records so far...
Collected 1463 records so far...
Collected 1573 records so far...
Collected 1681 records so far...
Collected 1807 records so far...
Collected 1952 records so far...
Collected 2116 records so far...
Collected 2266 records so far...
Collected 2445 records so far...
Collected 2630 records so far...
Collected 2847 records so far...
Collected 3020 records so far...
Collected 3176 records so far...
Collected 3339 records so far...
Collected 3485 records so far...
Collected 3615 records so far...
Collected 3751 records so far...
Collected 3901 records so far...
Collected 4047 reco

In [14]:
print(f"Total records collected: {len(all_asteroids)}")
print("Sample record:")
import pprint
pprint.pprint(all_asteroids[0])


Total records collected: 10000
Sample record:
{'absolute_magnitude_h': 21.11,
 'astronomical': 0.2027394097,
 'close_approach_date': '2024-05-07',
 'estimated_diameter_max_km': 0.3564839479,
 'estimated_diameter_min_km': 0.1594244681,
 'id': '2481457',
 'is_potentially_hazardous_asteroid': True,
 'miss_distance_km': 30329383.856177337,
 'miss_distance_lunar': 78.8656303733,
 'name': '481457 (2006 XD2)',
 'neo_reference_id': '2481457',
 'orbiting_body': 'Earth',
 'relative_velocity_kmph': 32286.8711562483}


In [15]:
import pandas as pd


In [16]:
df = pd.DataFrame(all_asteroids)
print(f"Shape: {df.shape}")
df.head()


Shape: (10000, 13)


,id,neo_reference_id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,2481457,2481457,481457 (2006 XD2),21.11,0.159424,0.356484,True,2024-05-07,32286.871156,0.202739,3.032938e+07,78.865630,Earth
1,3458580,3458580,(2009 HV44),18.92,0.437074,0.977327,False,2024-05-07,71938.546306,0.255989,3.829540e+07,99.579697,Earth
2,3752787,3752787,(2016 JE6),24.28,0.037030,0.082802,False,2024-05-07,67725.648904,0.160930,2.407474e+07,62.601665,Earth
3,3774003,3774003,(2017 HS),21.28,0.147420,0.329640,False,2024-05-07,69547.641447,0.273048,4.084737e+07,106.215594,Earth
4,3841905,3841905,(2019 JF7),24.97,0.026950,0.060262,False,2024-05-07,60342.098750,0.306347,4.582890e+07,119.169085,Earth


In [17]:
# Convert to float
df["absolute_magnitude_h"] = pd.to_numeric(df["absolute_magnitude_h"], errors='coerce')
df["estimated_diameter_min_km"] = pd.to_numeric(df["estimated_diameter_min_km"], errors='coerce')
df["estimated_diameter_max_km"] = pd.to_numeric(df["estimated_diameter_max_km"], errors='coerce')
df["relative_velocity_kmph"] = pd.to_numeric(df["relative_velocity_kmph"], errors='coerce')
df["astronomical"] = pd.to_numeric(df["astronomical"], errors='coerce')
df["miss_distance_km"] = pd.to_numeric(df["miss_distance_km"], errors='coerce')
df["miss_distance_lunar"] = pd.to_numeric(df["miss_distance_lunar"], errors='coerce')

# Convert boolean field
df["is_potentially_hazardous_asteroid"] = df["is_potentially_hazardous_asteroid"].astype(bool)

# Convert date field
df["close_approach_date"] = pd.to_datetime(df["close_approach_date"], format="%Y-%m-%d", errors='coerce')


In [18]:
# Drop rows with missing values in key columns
df_cleaned = df.dropna(subset=[
    "id", "neo_reference_id", "name", "absolute_magnitude_h",
    "estimated_diameter_min_km", "estimated_diameter_max_km",
    "relative_velocity_kmph", "miss_distance_km",
    "close_approach_date", "orbiting_body"
])

print(f"Shape after cleaning: {df_cleaned.shape}")
df_cleaned.head()


Shape after cleaning: (10000, 13)


,id,neo_reference_id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,2481457,2481457,481457 (2006 XD2),21.11,0.159424,0.356484,True,2024-05-07,32286.871156,0.202739,3.032938e+07,78.865630,Earth
1,3458580,3458580,(2009 HV44),18.92,0.437074,0.977327,False,2024-05-07,71938.546306,0.255989,3.829540e+07,99.579697,Earth
2,3752787,3752787,(2016 JE6),24.28,0.037030,0.082802,False,2024-05-07,67725.648904,0.160930,2.407474e+07,62.601665,Earth
3,3774003,3774003,(2017 HS),21.28,0.147420,0.329640,False,2024-05-07,69547.641447,0.273048,4.084737e+07,106.215594,Earth
4,3841905,3841905,(2019 JF7),24.97,0.026950,0.060262,False,2024-05-07,60342.098750,0.306347,4.582890e+07,119.169085,Earth


In [19]:
# Convert to list of tuples for SQL insertion
records_for_sql = list(df_cleaned.itertuples(index=False, name=None))

# Show sample row
print("Sample SQL-ready record:\n", records_for_sql[0])


Sample SQL-ready record:
 ('2481457', '2481457', '481457 (2006 XD2)', 21.11, 0.1594244681, 0.3564839479, True, Timestamp('2024-05-07 00:00:00'), 32286.8711562483, 0.2027394097, 30329383.856177337, 78.8656303733, 'Earth')


In [20]:
('1234567', '987654', 'AsteroidX', 22.5, 0.123, 0.456, True, datetime.date(2024, 1, 2), ...)


('1234567',
 '987654',
 'AsteroidX',
 22.5,
 0.123,
 0.456,
 True,
 datetime.date(2024, 1, 2),
 Ellipsis)

In [21]:
# Save cleaned DataFrame to a CSV file
df_cleaned.to_csv("cleaned_nasa_neo_data.csv", index=False)

print("✅ File 'cleaned_nasa_neo_data.csv' has been saved in your working directory.")


✅ File 'cleaned_nasa_neo_data.csv' has been saved in your working directory.


In [22]:
import os

print("📂 Current working directory:", os.getcwd())


📂 Current working directory: e:\New folder\Jupiter Notebook


In [26]:
import mysql.connector


In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime

# ✅ Step 1: Load the cleaned CSV
df = pd.read_csv("cleaned_nasa_neo_data.csv")

# Convert 'close_approach_date' to datetime object
df['close_approach_date'] = pd.to_datetime(df['close_approach_date'])

# ✅ Step 2: Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345678",  # 👉 Use your MySQL password if set (often blank for XAMPP)
    database="nasa_neo"
)

cursor = conn.cursor()

# ✅ Step 3: Define the insert query
insert_query = """
INSERT INTO neo_asteroids (
    id, neo_reference_id, name, absolute_magnitude_h,
    estimated_diameter_min_km, estimated_diameter_max_km,
    is_potentially_hazardous_asteroid, close_approach_date,
    relative_velocity_kmph, astronomical, miss_distance_km,
    miss_distance_lunar, orbiting_body
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# ✅ Step 4: Insert data row-by-row
for index, row in df.iterrows():
    data = (
        str(row['id']),
        str(row['neo_reference_id']),
        row['name'],
        float(row['absolute_magnitude_h']),
        float(row['estimated_diameter_min_km']),
        float(row['estimated_diameter_max_km']),
        bool(row['is_potentially_hazardous_asteroid']),
        row['close_approach_date'].date() if pd.notnull(row['close_approach_date']) else None,
        float(row['relative_velocity_kmph']),
        float(row['astronomical']),
        float(row['miss_distance_km']),
        float(row['miss_distance_lunar']),
        row['orbiting_body']
    )
    cursor.execute(insert_query, data)

# ✅ Step 5: Commit and close
conn.commit()
print("✅ All records inserted successfully into MySQL database.")

cursor.close()
conn.close()
  

✅ All records inserted successfully into MySQL database.
